# Homework #7 Solutions
### Portfolio Theory and Risk Management I

## Imports

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

## Data

In [29]:
df = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name=1).set_index('Date').dropna()

df.head()

,DP,EP,US10Y
Date,,,
1993-02-28,2.82,4.44,6.03
1993-03-31,2.77,4.41,6.03
1993-04-30,2.82,4.44,6.05
1993-05-31,2.81,4.38,6.16
1993-06-30,2.79,4.31,5.80


In [30]:
rf = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name=3).set_index('Date')

rf.head()

,US3M
Date,
1993-02-28,0.0025
1993-03-31,0.0025
1993-04-30,0.0025
1993-05-31,0.0026
1993-06-30,0.0026


In [31]:
GMO = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name=2).set_index('Date')

GMO_ex = GMO.dropna().subtract(rf['US3M'], axis=0).dropna()

GMO_ex.head()

,SPY,GMWAX
Date,,
1996-11-30,0.0687,0.0409
1996-12-31,-0.0282,-0.0156
1997-01-31,0.0575,0.0104
1997-02-28,0.0052,0.0179
1997-03-31,-0.0486,-0.0196


## 2 Analyzing GMO

**2.1:** Analyzing GMO's performance

In [32]:
def summary_stats(df, annual_fac = 12):
    stats_df = pd.DataFrame(data = None)
    stats_df['Mean'] = df.mean()*annual_fac
    stats_df['Vol'] = df.std()*np.sqrt(annual_fac)
    stats_df['Sharpe'] = df.mean()/df.std()*np.sqrt(annual_fac)
    
    return stats_df

In [33]:
summary_stats(GMO_ex.loc[:'2011', ['GMWAX']])

,Mean,Vol,Sharpe
GMWAX,0.0158,0.125,0.1266


In [34]:
summary_stats(GMO_ex.loc['2012':, ['GMWAX']])

,Mean,Vol,Sharpe
GMWAX,0.0593,0.0853,0.6952


In [35]:
summary_stats(GMO_ex[['GMWAX']])

,Mean,Vol,Sharpe
GMWAX,0.0329,0.1111,0.2964


GMO's performance 2012 onwards is much better than in the first part of the sample period. 

**2.2:** Analyzing GMO's tail risk

In [36]:
def tail_risk(df):
    tr_df = pd.DataFrame(data = None)
    tr_df['Min return'] = df.min()
    tr_df['VaR-5th'] = df.quantile(.05)
    cum_ret = (1 + df).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    tr_df['Max Drawdown'] = drawdown.min()
    
    return tr_df

In [37]:
tail_risk(GMO_ex.loc[:'2011', ['GMWAX']])

,Min return,VaR-5th,Max Drawdown
GMWAX,-0.1492,-0.0598,-0.4729


In [38]:
tail_risk(GMO_ex.loc[:'2011', ['SPY']])

,Min return,VaR-5th,Max Drawdown
SPY,-0.1656,-0.0802,-0.56


In [39]:
tail_risk(GMO_ex.loc['2012':, ['GMWAX']])

,Min return,VaR-5th,Max Drawdown
GMWAX,-0.1187,-0.0306,-0.1696


In [40]:
tail_risk(GMO_ex.loc['2012':, ['SPY']])

,Min return,VaR-5th,Max Drawdown
SPY,-0.1247,-0.0602,-0.1963


In [41]:
tail_risk(GMO_ex[['GMWAX']])

,Min return,VaR-5th,Max Drawdown
GMWAX,-0.1492,-0.0449,-0.4729


In [42]:
tail_risk(GMO_ex[['SPY']])

,Min return,VaR-5th,Max Drawdown
SPY,-0.1656,-0.0754,-0.56


(a) GMO has lower tail risk than SPY. Across all periods examined it has lower VaR, max drawdown, and minimum return.

(b) Yes, GMO's VaR improves notably in the second subsample, and drawdown is also much lower in the second subsample. 

**2.3:** GMO's Market Exposure

In [43]:
def reg_params(df, y_col, X_col, intercept = True, annual_fac=12):
    y = df[y_col]
    if intercept == True:
        X = sm.add_constant(df[X_col])
    else:
        X = df[X_col]
    
    model = sm.OLS(y, X, missing = 'drop').fit()
    reg_df = model.params.to_frame('Regression Parameters')
    reg_df.loc[r'$R^{2}$'] = model.rsquared
    
    if intercept == True:
        reg_df.loc['const'] *= annual_fac
    
    return reg_df

(a) Report the estimated alpha, $\beta$, and $R^{2}$

In [44]:
reg_params(GMO_ex.loc[:'2011'], 'GMWAX', 'SPY')

,Regression Parameters
const,-0.0058
SPY,0.5396
$R^{2}$,0.5071


In [45]:
reg_params(GMO_ex.loc['2012':], 'GMWAX', 'SPY')

,Regression Parameters
const,-0.0285
SPY,0.5683
$R^{2}$,0.7633


In [46]:
reg_params(GMO_ex, 'GMWAX', 'SPY')

,Regression Parameters
const,-0.0135
SPY,0.5461
$R^{2}$,0.5668


(b) Is GMWAX a low-beta strategy? Has that changed since the case?

While GMO has moderate exposure to the market and it's market beta is not very low, we can consider it a low-beta strategy. The beta is consistent across the subsamples so GMO's exposure to the market has not changed.   

(c) Does GMWAX provide alpha? Has that changed across the subsamples?

GMO does not provide alpha in either subsample as alpha is negative. 

## 3 Forecast Regressions

**3.1:** Forecast Regression Parameters

In [47]:
df = df.shift()
df['SPY'] = GMO['SPY']

df.head()

,DP,EP,US10Y,SPY
Date,,,,
1993-02-28,NaN,NaN,NaN,0.0107
1993-03-31,2.82,4.44,6.03,0.0224
1993-04-30,2.77,4.41,6.03,-0.0256
1993-05-31,2.82,4.44,6.05,0.0270
1993-06-30,2.81,4.38,6.16,0.0037


In [48]:
DP = reg_params(df, 'SPY', 'DP')

DP

,Regression Parameters
const,-0.0709
DP,0.0079
$R^{2}$,0.0068


In [49]:
EP = reg_params(df, 'SPY', 'EP')

EP

,Regression Parameters
const,-0.0488
EP,0.0029
$R^{2}$,0.0078


In [50]:
EP_DP_10Y = reg_params(df, 'SPY', ['EP','DP','US10Y'])

EP_DP_10Y 

,Regression Parameters
const,-0.1135
EP,0.0025
DP,0.0064
US10Y,-0.0013
$R^{2}$,0.0150


**3.2:** For each of the three regressions, let’s try to utilize the resulting forecast in a trading strategy.
> Factors are already shifted

In [51]:
w_DP = 100 * (DP.loc['const'][0]/12 + DP.loc['DP'][0] * df['DP'])

r_DP = (w_DP * df['SPY']).dropna()

w_EP = 100 * (EP.loc['const'][0]/12 + EP.loc['EP'][0] * df['EP'])

r_EP = (w_EP * df['SPY']).dropna()

w_3fac = 100 * (EP_DP_10Y.loc['const'][0]/12 + EP_DP_10Y.loc['EP'][0] * df['EP']\
                                             + EP_DP_10Y.loc['DP'][0] * df['DP']\
                                             + EP_DP_10Y.loc['US10Y'][0] * df['US10Y'])

r_3fac = (w_3fac * df['SPY']).dropna()

In [52]:
def summary_stats_bm(series, bm, annual_fac=12):
    ss_df = pd.DataFrame(data = None, index = ['Summary Stats'])
    ss_df['Mean'] = series.mean() * annual_fac
    ss_df['Vol'] = series.std() * np.sqrt(annual_fac)
    ss_df['Sharpe (Mean/Vol)'] = ss_df['Mean'] / ss_df['Vol']
    
    y = series
    X = sm.add_constant(bm.loc[series.index])
    reg = sm.OLS(y,X).fit().params
    ss_df[r'$\alpha$'] = reg[0] * annual_fac
    ss_df[r'$\beta^{SPY}$'] = reg[1]
    
    cum_ret = (1 + series).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    ss_df['Max Drawdown'] = drawdown.min()
    
    return round(ss_df, 4)

In [53]:
summary_stats_bm(r_DP, df[['SPY']])

,Mean,Vol,Sharpe (Mean/Vol),$\alpha$,$\beta^{SPY}$,Max Drawdown
Summary Stats,0.118,0.1511,0.7809,0.0132,0.9407,-0.6415


In [54]:
summary_stats_bm(r_EP, df[['SPY']])

,Mean,Vol,Sharpe (Mean/Vol),$\alpha$,$\beta^{SPY}$,Max Drawdown
Summary Stats,0.12,0.1321,0.9081,0.0311,0.798,-0.385


In [55]:
summary_stats_bm(r_3fac, df[['SPY']])

,Mean,Vol,Sharpe (Mean/Vol),$\alpha$,$\beta^{SPY}$,Max Drawdown
Summary Stats,0.1354,0.1499,0.9032,0.0408,0.8491,-0.5193


**3.3:** GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. Let’s consider risk characteristics.

(a) For both strategies, the market, and GMO, calculate the monthly VaR for π = .05. Just use the quantile of the historic data for this VaR calculation.

In [56]:
VaR = pd.DataFrame([r_DP.quantile(.05), r_EP.quantile(.05), r_3fac.quantile(.05), 
                    df['SPY'].quantile(.05), 
                    GMO['GMWAX'].quantile(.05)],
                   index = ['DP Strat','EP Strat','3-factor Strat','SPY','GMO'], 
                   columns = ['5% VaR'])

VaR

,5% VaR
DP Strat,-0.0579
EP Strat,-0.0549
3-factor Strat,-0.0645
SPY,-0.0694
GMO,-0.0444


(b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

In [338]:
summary_stats(r_DP.to_frame('DP Strat').loc['2000':'2011'])

,Mean,Vol,Sharpe
DP Strat,0.0365,0.1821,0.2005


In [339]:
summary_stats(r_EP.to_frame('EP Strat').loc['2000':'2011'])

,Mean,Vol,Sharpe
EP Strat,0.0367,0.1407,0.261


In [340]:
summary_stats(r_3fac.to_frame('3-factor Strat').loc['2000':'2011'])

,Mean,Vol,Sharpe
3-factor Strat,0.0597,0.1601,0.3729


In [341]:
summary_stats(rf.loc['2000':'2011'])

,Mean,Vol,Sharpe
US3M,0.0231,0.0058,3.9866


All the dynamic strategies outperform the risk-free rate during this period. 

(c) Based on the regression estimates, in how many periods do we estimate a negative risk premium?

In [342]:
r_df = r_3fac.to_frame('3-factor Strat')
r_df['DP Strat'] = r_DP
r_df['EP Strat'] = r_EP
r_df['rf'] = rf['US3M']

df_riskprem = pd.DataFrame(data=None, index=[r'% of periods underperforming $r^{f}$'])
for col in r_df.columns[:3]:
    df_riskprem[col] = len(r_df[r_df[col] < r_df['rf']])/len(r_df) * 100
    
df_riskprem

,3-factor Strat,DP Strat,EP Strat
% of periods underperforming $r^{f}$,37.5,36.0465,37.2093


(d) Do you believe the dynamic strategy takes on extra risk??

No, judging by the tail risk metrics and volatility of the dynamic strategies compared to SPY it does not seem like these strategies take on extra risk on the whole.

However, we must keep in mind that the strategies are dependent on running regressions with very little prediction power, so badly estimated parameters could lead to terrible performance. This is not evident in terms of very high volatility and tail risk in our backtesting period though. 

## 4 Out-of-Sample Forecasting 

In [131]:
def OOS_r2(df, factors, start):
    y = df['SPY']
    X = sm.add_constant(df[factors])

    forecast_err, null_err = [], []

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            forecast_err.append(reg_predict - actual)
            null_err.append(null_forecast - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    
    return 1 - RSS/TSS

In [132]:
EP_OOS_r2 = OOS_r2(df, ['EP'], 60)

print('EP OOS R-squared: ' + str(round(EP_OOS_r2, 4)))

EP OOS R-squared: -0.0092


**4.1:** Did this forecasting strategy produce a positive OOS r-squared?

No, the $R^{2}$ value is negative.

**4.2:**

In [133]:
def OOS_strat(df, factors, start, weight):
    returns = []
    y = df['SPY']
    X = sm.add_constant(df[factors])

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            pred = reg.predict(X.iloc[[i]])
            w = pred * weight
            returns.append((df.iloc[i]['SPY'] * w)[0])

    df_strat = pd.DataFrame(data = returns, index = df.iloc[-(len(returns)):].index, columns = ['Strat Returns'])
    return df_strat

In [134]:
OOS_EP = OOS_strat(df, ['EP'], 60, 100)

OOS_EP.head()

,Strat Returns
Date,
1998-02-28,0.0512
1998-03-31,0.0456
1998-04-30,0.0143
1998-05-31,-0.0225
1998-06-30,0.0351


In [135]:
summary_stats_bm(OOS_EP['Strat Returns'], GMO[['SPY']])

,Mean,Vol,Sharpe (Mean/Vol),$\alpha$,$\beta^{SPY}$,Max Drawdown
Summary Stats,0.0889,0.1632,0.5445,0.0405,0.5095,-0.5837


**4.3:**

(a)

In [136]:
VaR_OOS = pd.DataFrame([OOS_EP['Strat Returns'].quantile(.05),  
                    df['SPY'].quantile(.05), 
                    GMO['GMWAX'].quantile(.05)],
                   index = ['EP Strat','SPY','GMO'], 
                   columns = ['5% VaR'])

VaR_OOS

,5% VaR
EP Strat,-0.0668
SPY,-0.0694
GMO,-0.0444


(b)

In [137]:
summary_stats_bm(OOS_EP.loc['2000':'2011']['Strat Returns'], GMO[['SPY']])

,Mean,Vol,Sharpe (Mean/Vol),$\alpha$,$\beta^{SPY}$,Max Drawdown
Summary Stats,0.0388,0.1959,0.1979,0.0333,0.2994,-0.5837


In [138]:
summary_stats(rf.loc['2000':'2011'])

,Mean,Vol,Sharpe
US3M,0.0231,0.0058,3.9866


The dynamic strategy barely overperforms the risk-free rate in terms of premium. 

(c)

In [139]:
r_df_OOS = OOS_EP.rename(columns={"Strat Returns": "EP Strat"})
r_df_OOS['rf'] = rf['US3M']

df_riskprem2 = pd.DataFrame(data=None, index=[r'% of periods underperforming $r^{f}$'])
for col in r_df_OOS.columns[:3]:
    df_riskprem2[col] = len(r_df_OOS[r_df_OOS[col] < r_df_OOS['rf']])/len(r_df_OOS) * 100
    
df_riskprem2

,EP Strat,rf
% of periods underperforming $r^{f}$,37.5439,0.0


(d)

The dynamic strategy tends to have worse risk metrics than SPY so it seems this strategy does take on extra risk.